In [ ]:
from wordcloud import WordCloud

from analytics.read import *
from analytics.preprocess import *
from analytics.analysis import *

# Input benchmark and target dataset list

In [ ]:
dict_target = dict()

dict_target['benchmark'] = ['일취월장', '완공', '뼈아대'] 
dict_target['일취월장'] = ['일취월장'] 
dict_target['완공'] = ['완공'] 
dict_target['뼈아대'] = ['뼈아대'] 

dict_target['personal'] = ['Daily_Report', 'Weekly_Report', 'Monthly_Report', '글쓰기'] 
dict_target['멘토_브런치'] = ['멘토_브런치']
dict_target['신박사_브런치'] = ['신박사_브런치']
dict_target['신영준_페이스북'] = ['신영준_페이스북']
dict_target['연예_기사'] = ['연예_기사'] 

In [ ]:
from glob import glob

files = glob('text/*/*.txt')
names = {file.split('/')[1] for file in files}

dict_target = dict()

for name in names:
    dict_target[name] = [str(i) for i in range(1, 8)] 

# dict_target['경규승'] = [str(i) for i in range(1, 8)] 
# dict_target['경규승'] = [str(i) for i in range(1, 8)] 

# Data Read

## .txt 읽은 dict

In [ ]:
dict_target.keys()

In [ ]:
dict_strings = get_dict_strings(dict_target)

In [ ]:
dict_merged_strings = get_dict_merged_strings(dict_strings)

In [ ]:
# dict_merged_strings['이정애']

## .txt을 읽은 dict -> noun의 list로

In [ ]:
dict_nouns = get_nouns_from_topics(dict_merged_strings, komoran)

# Data Explore

In [ ]:
for key in dict_nouns.keys():
    print('# of nouns in {0}: {1}'.format(key, len(dict_nouns[key])))

In [ ]:
%matplotlib agg

In [ ]:
for category in dict_nouns.keys():
    plt.figure(figsize=(20, 6))
    plt.title(category)

    # 30개 noun 까지만 보여줌.
    Text(dict_nouns[category], name=category).plot(70)
    plt.savefig(f'output/{category}_original_graph.png')
    # plt.show()
    # plt.close()

-> 지수적으로 떨어지는게 자연적인 모양. 오히려 지수적으로 떨어지지 않으면 데이터가 부족한 것이라 판단.

## noun 빈도수를 포함한 dict로

In [ ]:
dict_frequency = get_nouns_frequency(dict_nouns)

# Data Preprocess

의미 부족한 단어 제거 (10 단어 이상의 경우만 체크함)

In [ ]:
delete_words_benchmark = ['것', '수', '때', '때문', '경우', '필요', '가지', '정도', '뿐', '년', 
                          '등', '데' '개', '대부분', '번', '점', '관련', '전', '중', '자', '분', 
                          '만', '게', '동안', '뜻', '곳', '언가']
delete_words_personal = ['것', '수', '때', '안', '때문', '건', '필요', '곳', '번', '전', '중', '정도',
                         '년', '거', '점', '.so', '라고', '부분', '구체', '뿐', '자체', '가지', '밖',
                         '날', '분', '.com', '데', '개', '의', '을', '자', '션', '한', '회', '줄', '영',
                         '만', '명', '적', '시']
delete_words_huge_portion = ['사람', '자신', '일', '말', '생각', '책', '페이지', '속', '경', '듯', '조금']
delete_words = delete_words_benchmark + delete_words_personal + delete_words_huge_portion

In [ ]:
dict_nouns_ppd = get_preprocessed_nouns(dict_nouns, delete_words)

In [ ]:
for category in dict_nouns_ppd.keys():
    plt.figure(figsize=(20, 6))
    plt.title(category)
    Text(dict_nouns_ppd[category], name=category).plot(70)
    plt.savefig(f'output/{category}_ppd_graph.png')
    # plt.show()

In [ ]:
dict_frequency_ppd = get_preprocessed_nouns_frequency(dict_frequency, delete_words)

In [ ]:
import PIL.Image
mask = np.array(PIL.Image.open("vocation.png"))

In [ ]:
font_path = '/Users/gs/Library/Fonts/NanumBarunGothic.otf'
for category in dict_frequency_ppd.keys():
    # wc = WordCloud(width = 1500, height = 1500, background_color="white", font_path=font_path)
    wc = WordCloud(
                   # width=800,
                   # height=300,
                   background_color="white",
                   mask=mask,
                   font_path=font_path,
                   max_words=150,
                   max_font_size=150,
                   random_state=42,
                   contour_width=0,
                   contour_color='steelblue',
                   colormap='tab20')
    # plt.figure(figsize=(10,60))
    plt.title(category, fontsize=15)
    plt.tight_layout(pad=0)
    plt.imshow(wc.generate_from_frequencies(dict_frequency_ppd[category]))
    plt.axis("off")
    plt.savefig(f'output/{category}_ppd_wc.png')
    # plt.show()

In [ ]:
# font_path = '/Users/gs/Library/Fonts/NanumBarunGothic.otf'
# for category in list(dict_frequency_ppd.keys())[:1]:
#     # wc = WordCloud(width = 1500, height = 1500, background_color="white", font_path=font_path)
#     wc = WordCloud(width=1500,
#                    height=1500,
#                    background_color="white",
#                    font_path=font_path,
#                    max_words=150,
#                    max_font_size=150,
#                    random_state=42,
#                    contour_width=3,
#                    # contour_color='steelblue',
#                    colormap='Blues')
#     plt.figure( figsize=(20,10))
#     plt.title(category, fontsize=25)
#     plt.imshow(wc.generate_from_frequencies(dict_frequency_ppd[category]))
#     plt.axis("off")
#     plt.savefig(f'output/{key}_ppd_wc.png')
#     # plt.show()

# Analysis

## sse_score

In [ ]:
pd.options.display.max_columns = 40

In [ ]:
df_sse = pd.DataFrame()
for category_a in dict_frequency_ppd.keys():
    for category_b in dict_frequency_ppd.keys():
        df_sse = df_sse.append([[category_a, category_b,
                                get_score(
                                    dict_frequency_ppd[category_a], dict_frequency_ppd[category_b], 'sse')]],
                               ignore_index=True)
df_sse.columns = ['category_a', 'category_b', 'score']
df_sse.pivot(index='category_a', columns='category_b', values='score')
# pd.pivot_table(df_sse, index='category_a', columns='category_b', values='score')

In [ ]:
df_sse.pivot(index='category_a', columns='category_b', values='score')

In [ ]:
# log_sse -> log는 의도와는 좀 다르게 나오는 것 같음. 전처리 의도는 맞는데 왜 그럴까?
df_sse = pd.DataFrame()
for category_a in dict_frequency_ppd.keys():
    for category_b in dict_frequency_ppd.keys():
        df_sse = df_sse.append([[category_a, category_b,
                                get_score(
                                    dict_frequency_ppd[category_a], dict_frequency_ppd[category_b], 'log_sse')]],
                               ignore_index=True)
df_sse.columns = ['category_a', 'category_b', 'score']

In [ ]:
df_sse.pivot(index='category_a', columns='category_b', values='score')

In [ ]:
# for column order
df_sse_pivot = df_sse.pivot(index='category_a', columns='category_b', values='score')
list_order = ['benchmark',
              'personal',
              '신박사_브런치',
              '신영준_페이스북',
              '멘토_브런치',
              '연예_기사',
              '뼈아대',
              '일취월장',
              '완공']
df_sse_pivot[list_order].reindex(index=list_order)

## LDA

### decide # of topics

In [ ]:
documents = []
category = 'benchmark'
for sentence in [s for s in dict_merged_strings[category].split("\n") if s]:
    documents.append(' '.join([s for s in komoran.nouns(sentence) if s]))

In [ ]:
n_topics = [i for i in range(2, 25)]
learning_decays = [round(0.35 + 0.05 * i, 2) for i in range(0, 4)]

In [ ]:
# warning 은 .com , .so 제거하는 과정에서 발생.

list_perplexity = []

for ld in learning_decays:
    for n_components in n_topics:
        tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words=delete_words)
        tf = tf_vectorizer.fit_transform(documents)
        tf_feature_names = tf_vectorizer.get_feature_names()

        # Run LDA
        lda = LatentDirichletAllocation(learning_decay = ld,
                                        n_components=n_components, max_iter=5, 
                                        learning_method='online', learning_offset=50.,random_state=0,
                                        doc_topic_prior=0.1, topic_word_prior=0.01,
                                       ).fit(tf)
        # perplexity
        tf_perplexity = lda.perplexity(tf)
        
        list_perplexity.append([n_components, ld, tf_perplexity])

df_perplexity = pd.DataFrame(list_perplexity, columns=['n_topics', 'learning_decay', 'perplexity'])

In [ ]:
# reference: https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/
plt.figure(figsize=(12, 8))

for i in range(len(learning_decays)):
    plt.plot(n_topics, df_perplexity[df_perplexity.learning_decay==learning_decays[i]].perplexity, 
             label=learning_decays[i])

plt.title("Choosing Optimal LDA Model")
plt.xlabel("Num Topics")
plt.ylabel("perplexity")
plt.legend(title='Learning decay', loc='best')
plt.show()

#### cluster 숫자 결정  
decided_n_components = 21  
decided_ld = 0.45

### topic clustering

In [ ]:
# 클러스터 수, learning_decay 결정
decided_n_components = 21
decided_ld = 0.45

In [ ]:
lda, lda_tf_feature_names = get_lda_model(dict_merged_strings, decided_ld, decided_n_components, 
                                          delete_words, category='benchmark')

In [ ]:
# display
no_top_words = 10
display_topics(lda, lda_tf_feature_names, no_top_words)

In [ ]:
df_topic_clustering = get_lda_topic_clustering(lda, lda_tf_feature_names)
df_topic_clustering

In [ ]:
# 토픽별 단어 숫자
df_topic_clustering.topic_words.apply(lambda x: len(x))

In [ ]:
# label 붙이기.
df_topic_clustering_label = df_topic_clustering.copy()
df_topic_clustering_label['topic_label'] = [
    '계획', 
    '학습', 
    '능력', 
    '마음',
    '세상',
    '정보와 아이디어',
    '믿음',
    '습관',
    '성공과 도전',
    '공부',
    '행복',
    '회사내 태도',
    '실력',
    '기억',
    '목표',
    '결과',
    '창의',
    '행동',
    '호기심',
    '조직',
    '메타인지',]

In [ ]:
df_topic_clustering_label[['topic_label', 'topic_words']]

## topic sse score

In [ ]:
df_topic_scores = pd.DataFrame()
for category in ['personal', '뼈아대', '일취월장', '완공', '신박사_브런치', '신영준_페이스북', '멘토_브런치', '연예_기사']:
    df_topic_score = get_topic_scores(
        dict_frequency_ppd['benchmark'], dict_frequency_ppd[category], df_topic_clustering)
    df_topic_score.columns = ['{}_score'.format(category)]
    
    df_topic_scores = pd.concat([df_topic_scores, df_topic_score], axis=1)

In [ ]:
df_topic_label_score = pd.merge(df_topic_clustering_label[['topic_label']], df_topic_scores,
                                left_index=True, right_index=True)
df_topic_label_score.reset_index(inplace=True)
df_topic_label_score.set_index(['topic_index', 'topic_label'], inplace=True)

df_topic_label_score

In [ ]:
n_top_bottom_topic = 3

In [ ]:
df_top_n_topics = pd.DataFrame()
for column in df_topic_label_score.columns:
    df_top_n_topics = df_top_n_topics.append(
        [['_'.join(column.split('_')[:-1]),
         df_topic_label_score.reset_index().nlargest(
             n_top_bottom_topic, column).topic_label.to_list(),
         df_topic_label_score.reset_index().nlargest(
             n_top_bottom_topic, column)[column].mean()]], ignore_index=True)
df_top_n_topics.columns = ['category', 'strong_topics', 'strong_topics_mean_scores']
df_top_n_topics.set_index('category', inplace=True)

In [ ]:
df_bottom_n_topics = pd.DataFrame()
for column in df_topic_label_score.columns:
    df_bottom_n_topics = df_bottom_n_topics.append(
        [['_'.join(column.split('_')[:-1]),
         df_topic_label_score.reset_index().nsmallest(
             n_top_bottom_topic, column).topic_label.to_list(),
         df_topic_label_score.reset_index().nsmallest(
             n_top_bottom_topic, column)[column].mean()]], ignore_index=True)
df_bottom_n_topics.columns = ['category', 'weak_topics', 'weak_topics_mean_scores']
df_bottom_n_topics.set_index('category', inplace=True)

In [ ]:
df_strong_weak_topics = pd.merge(df_top_n_topics, df_bottom_n_topics, 
                                 left_index=True, right_index=True)
df_strong_weak_topics